In [1]:
from data_processing_soc import *

from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import optuna

import numpy as np
from tqdm import tqdm
import copy

##########################################################
# Data loading
##########################################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "../01_Datenaufbereitung/Output/Calculated/"
all_data = load_data(data_dir)

df_25 = (all_data
         .groupby('cell_id', group_keys=False)  
         .apply(lambda x: x.iloc[:int(0.25 * len(x))])
         .reset_index(drop=True)
        )

train_df, val_df, test_df = split_data(df_25, train=13, val=1, test=1)
train_scaled, val_scaled, test_scaled = scale_data(train_df, val_df, test_df)


Found 15 parquet files


Processing cells:   0%|          | 0/15 [00:00<?, ?cell/s]

Processing C01 ...


Processing cells:   7%|▋         | 1/15 [00:23<05:25, 23.22s/cell]

Processing C03 ...


...

Processing C27 ...


Processing cells:  93%|█████████▎| 14/15 [02:38<00:11, 11.59s/cell]

Processing C29 ...


Processing cells: 100%|██████████| 15/15 [02:47<00:00, 11.16s/cell]


MemoryError: Unable to allocate 1.55 GiB for an array with shape (1, 207931524) and data type float64

In [3]:
print(f"Train: {len(train_df)}  |  Val: {len(val_df)}  |  Test: {len(test_df)}")


NameError: name 'train_df' is not defined

In [5]:
class SequenceDataset(Dataset):
    """
    (X[t], y[t]) mit seq_len Zeitschritten.
    - X[t] = [Voltage, Current, SOC] für t..t+seq_len-1
    - y[t] = SOC an (t + seq_len)
    """
    def __init__(self, df, seq_len=60):
        # Hier NICHT SOC skalieren, da er bereits in 0..1 liegt
        self.seq_len = seq_len
        data_array = df[["SOC_ZHU", "Current[A]",  "Voltage[V]", "Temperature[°C]"]].values
        self.data = data_array

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x_seq = self.data[idx : idx + self.seq_len]     # shape (seq_len, 3)
        y_val = self.data[idx + self.seq_len, 2]        # SOC in Spalte 2
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

# Erstellen der Datasets
seq_length = 60
train_dataset = SequenceDataset(train_scaled, seq_len=seq_length)
val_dataset   = SequenceDataset(val_scaled,   seq_len=seq_length)
test_dataset  = SequenceDataset(test_scaled,  seq_len=seq_length)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,  drop_last=True)
val_loader   = DataLoader(val_dataset,   batch_size=128, shuffle=False, drop_last=True)
test_loader  = DataLoader(test_dataset,  batch_size=128, shuffle=False, drop_last=True)

In [7]:
class LSTMSOCModel(nn.Module):
    def __init__(self, input_size=3, hidden_size=32, num_layers=1, batch_first=True):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=batch_first
        )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x: (batch_size, seq_len, 3)
        lstm_out, (h_n, c_n) = self.lstm(x)
        last_out = lstm_out[:, -1, :]  # (batch_size, hidden_size)
        soc_pred = self.fc(last_out)
        return soc_pred.squeeze(-1)    # (batch_size,)

model = LSTMSOCModel(input_size=3, hidden_size=32, num_layers=2, batch_first=True)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)


In [1]:
print(f"Train: {len(df_train)}  |  Val: {len(df_val)}  |  Test: {len(df_test)}")


NameError: name 'df_train' is not defined